<a href="https://colab.research.google.com/github/andyarnell/sepal_mgci/blob/master/Global_calculation_AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

#### Clone SDG 15.4.2 github repo into colab instance

In [4]:
!git clone https://github.com/andyarnell/sepal_mgci.git

Cloning into 'sepal_mgci'...
remote: Enumerating objects: 2800, done.
remote: Counting objects: 100% (738/738), done.
remote: Compressing objects: 100% (290/290), done.
remote: Total 2800 (delta 467), reused 692 (delta 447), pack-reused 2062
Receiving objects: 100% (2800/2800), 4.88 MiB | 18.93 MiB/s, done.
Resolving deltas: 100% (1743/1743), done.


In [13]:
import ee
# !pip install pygaul # if not already installed
import pygaul
import pandas as pd
from pathlib import Path
from openpyxl.utils import get_column_letter
from openpyxl.styles import Alignment

# import component.parameter.module_parameter as param
# from component.scripts.gee import reduce_regions
# from component.scripts.scripts import get_a_years, map_matrix_to_dict, parse_result
# from component.scripts.scripts import read_from_csv
# from component.scripts import sub_a as sub_a
# from component.scripts import sub_b as sub_b
# from component.scripts import mountain_area as mntn


# ee.Initialize()

In [ ]:
# Here I'm using pygaul to get the AOI based on the name
# but you can use any feature collection from Earth Engine
# just replace the aoi with your feature collection.

aoi_name = "Risaralda"
aoi = pygaul.Items(aoi_name)

## SUB INDICATOR A

In [ ]:
# Read the default land cover remapping table and convert it to a dictionary
default_map_matrix = map_matrix_to_dict(param.LC_MAP_MATRIX)

In [ ]:
# For SUB_A indicator, we need to set the following structure
a_years = {
    1: {"asset": "users/amitghosh/sdg_module/esa/cci_landcover/2000", "year": 2000},
    2: {"year": 2003, "asset": "users/amitghosh/sdg_module/esa/cci_landcover/2003"},
    3: {"year": 2007, "asset": "users/amitghosh/sdg_module/esa/cci_landcover/2007"},
    4: {"year": 2010, "asset": "users/amitghosh/sdg_module/esa/cci_landcover/2010"},
}

# Just extract the years from the a_years dictionary
single_years = [y["year"] for  y in a_years.values()]

[2000, 2003, 2007, 2010]

In [ ]:
process = ee.FeatureCollection([
    ee.Feature(
        None,
        reduce_regions(
            aoi,
            remap_matrix=default_map_matrix,
            rsa=False,
            dem=param.DEM_DEFAULT,
            lc_years= year,
            transition_matrix=False
        )
    ).set("process_id", year[0]["year"])
 for year in get_a_years(a_years)
])

In [ ]:
task_name = "Risaralda_sub_a_3"

In [ ]:
task = ee.batch.Export.table.toDrive(
    **{
        "collection": process,
        "description": task_name,
        "fileFormat": "CSV",
        "selectors": [
            "process_id",
            "sub_a",
        ],
    }
)
task.start()

# Read, process, and create report tables

In [ ]:
# Manually check your earth engine task status, once the task is completed, run the next cell
# according to the task name you have set above.

In [ ]:
results_file_path = Path().home()/ "downloads"/ "Risaralda_sub_a_3.csv"
report_folder = Path().home() / "downloads" / "Risaralda"
report_folder.mkdir(exist_ok=True)

output_name = str(
    Path(report_folder, "report_name.xlsx")
)

In [ ]:
# This line will read the results from the CSV file and parse it to a dictionary
dict_results = read_from_csv(results_file_path)

In [ ]:
# Some extra details that have to be manually added to the report
details = {
    "geo_area_name" : " ",
    "ref_area" : " ",
    "source_detail" : " ",
}

In [ ]:
# Following lines will generate the reports for the sub_a and mtn indicators

sub_a_reports = []
mtn_reports = []
for year in single_years:
    print(f"Reporting {year} for sub_a")
    parsed_df = parse_result(dict_results[year]["sub_a"], single=True)

    sub_a_reports.append(sub_a.get_reports(parsed_df, year, **details))

    print(f"Reporting {year} for mtn")
    mtn_reports.append(mntn.get_report(parsed_df, year, **details))


Reporting 2000 for sub_a
Reporting 2000 for mtn
Reporting 2003 for sub_a
Reporting 2003 for mtn
Reporting 2007 for sub_a
Reporting 2007 for mtn
Reporting 2010 for sub_a
Reporting 2010 for mtn


In [ ]:
mtn_reports_df = pd.concat(mtn_reports)

# sub a reports
er_mtn_grnvi_df = pd.concat([report[0] for report in sub_a_reports])
er_mtn_grncov_df = pd.concat([report[1] for report in sub_a_reports])


In [ ]:
# This will create the excel file with the reports

with pd.ExcelWriter(output_name) as writer:
    mtn_reports_df.to_excel(writer, sheet_name="Table1_ER_MTN_TOTL", index=False)
    er_mtn_grncov_df.to_excel(
        writer, sheet_name="Table2_ER_MTN_GRNCOV", index=False
    )
    er_mtn_grnvi_df.to_excel(writer, sheet_name="Table3_ER_MTN_GRNCVI", index=False)

    for sheetname in writer.sheets:
        worksheet = writer.sheets[sheetname]
        for col in worksheet.columns:
            max_length = 0
            column = col[0]
            for cell in col:
                try:
                    if len(str(cell.value)) > max_length:
                        max_length = len(cell.value)
                except:
                    pass
            adjusted_width = max(max_length, len(str(column.value))) + 4
            worksheet.column_dimensions[get_column_letter(column.column)].width = (
                adjusted_width
            )

            # Align "obs_value" column to the right
            if "OBS" in column.value:
                for cell in col:
                    cell.alignment = Alignment(horizontal="right")